In [1]:
pip install gtts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!wget "https://pjreddie.com/media/files/yolov3.weights"

!wget "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg"

!wget "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"

--2023-04-17 17:58:57--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  36.4MB/s    in 6.7s    

2023-04-17 17:59:04 (35.2 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2023-04-17 17:59:04--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      


In [9]:
import cv2
import numpy as np
from gtts import gTTS
from IPython.display import Audio

# Load the image
img = cv2.imread('7.jpg')

# Load the YOLOv3 object detection model
net = cv2.dnn.readNetFromDarknet('yolov3.cfg','yolov3.weights')

# Load the COCO class names
classes = []
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Define the names of the output layers
output_layers = net.getUnconnectedOutLayersNames()

# Define the minimum confidence threshold for detections
conf_threshold = 0.3

# Define the non-maximum suppression threshold for overlapping boxes
nms_threshold = 0.3

if(type(img)==np.ndarray):
  # Increase the image size
  img = cv2.resize(img, (608, 608))

  # Preprocess the image
  blob = cv2.dnn.blobFromImage(img, 1/255.0, (608, 608), swapRB=True, crop=False)

  # Pass the image through the network
  net.setInput(blob)
  outs = net.forward(output_layers)

  # Extract the object labels and bounding boxes from the network outputs
  class_ids = []
  confidences = []
  boxes = []
  for out in outs:
      for detection in out:
          scores = detection[5:]
          class_id = np.argmax(scores)
          confidence = scores[class_id]
          if confidence > conf_threshold:
              center_x = int(detection[0] * img.shape[1])
              center_y = int(detection[1] * img.shape[0])
              width = int(detection[2] * img.shape[1])
              height = int(detection[3] * img.shape[0])
              left = int(center_x - width / 2)
              top = int(center_y - height / 2)
              class_ids.append(class_id)
              confidences.append(float(confidence))
              boxes.append([left, top, width, height])

  # Apply non-maximum suppression to remove overlapping boxes
  indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
  labels = []
  for i in indices:
      i = int(i)
      label = class_ids[i]
      if label not in labels:
        labels.append(label)

  # Print the object labels
  label_name=[]
  if len(indices) > 0:
      print('This image contains the following objects:')
      for i in indices.flatten():
          label = str(classes[class_ids[i]])
          if label not in label_name:
            print(label)
            label_name.append(label)
  else:
      print('No objects were detected in this image.')

  # Generate the audio description
  if len(indices) != 0:
    description = 'This image contains '
    for i, label in enumerate(label_name):
      if i == 0:
        description += label
      elif i == len(labels) - 1:
        description += ' and ' + label
      else:
        description += ', ' + label

    # Convert the description to speech and save it
    tts = gTTS(description)
    tts.save('description.mp3')

    # Play the audio
    wn = Audio('description.mp3', autoplay=True)
    display(wn)


else:
  print("No image with this name found!")

This image contains the following objects:
person
bus
chair
